In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
#importing file
df = pd.read_excel('../data/proshares_analysis_data.xlsx', sheet_name='hedge_fund_series', index_col=0)
df.head()

,HFRIFWI Index,MLEIFCTR Index,MLEIFCTX Index,HDG US Equity,QAI US Equity
2011-08-31,-0.032149,-0.025588,-0.025689,-0.027036,-0.006489
2011-09-30,-0.038903,-0.032414,-0.032593,-0.032466,-0.022141
2011-10-31,0.026858,0.043593,0.043320,0.050531,0.025239
2011-11-30,-0.013453,-0.012142,-0.012431,-0.028608,-0.007964
2011-12-31,-0.004479,0.001938,0.001796,0.012875,0.001822


In [27]:
#calculating summary statistics
summary_stats = df.describe().iloc[1:3]
summary_stats.loc['mean'] *= 12
summary_stats.loc['std'] *= 12**0.5
summary_stats.loc['sharpe'] = summary_stats.loc['mean'] / summary_stats.loc['std']
summary_stats

,HFRIFWI Index,MLEIFCTR Index,MLEIFCTX Index,HDG US Equity,QAI US Equity
mean,0.048295,0.036286,0.034605,0.024812,0.025653
std,0.059476,0.056136,0.055980,0.058408,0.050009
sharpe,0.812004,0.646398,0.618172,0.424805,0.512971


In [30]:
#calculating further statistics
summary_stats.loc['skewness'] = df.skew()
summary_stats.loc['kurtosis'] = df.kurtosis()
summary_stats.loc['VaR (.05)'] = df.quantile(.05)
summary_stats.loc['CVaR (.05)'] = df.apply(lambda x: x[x <= x.quantile(.05)].mean())
summary_stats

,HFRIFWI Index,MLEIFCTR Index,MLEIFCTX Index,HDG US Equity,QAI US Equity
mean,0.048295,0.036286,0.034605,0.024812,0.025653
std,0.059476,0.056136,0.055980,0.058408,0.050009
sharpe,0.812004,0.646398,0.618172,0.424805,0.512971
skewness,-0.963318,-0.265394,-0.251279,-0.254201,-0.434775
kurtosis,5.763074,1.631580,1.595863,1.759250,1.605934
VaR (.05),-0.024293,-0.028189,-0.028095,-0.030932,-0.017895
CVaR (.05),-0.037478,-0.035857,-0.035816,-0.037575,-0.032695


In [40]:
#calculating max drawdown
cumulative_returns = (1+df).cumprod()
runningmax = cumulative_returns.cummax()
drawdown = (cumulative_returns - runningmax)/runningmax

def maxdrawdownfunc(series):
    maxdrawdown = drawdown[series.name].min()
    peakdate = runningmax[series.name].idxmax()
    troughdate = drawdown[series.name].idxmin()
    recoverydate = cumulative_returns[series.name][troughdate:].ge(runningmax[series.name].loc[troughdate]).idxmax()

    return pd.Series({
        'Max_Drawdown': maxdrawdown,
        'Peak_Date': peakdate,
        'Trough_Date': troughdate,
        'Recovery_Date': recoverydate
    })

max_drawdown_stats = df.apply(maxdrawdownfunc)
max_drawdown_stats

,HFRIFWI Index,MLEIFCTR Index,MLEIFCTX Index,HDG US Equity,QAI US Equity
Max_Drawdown,-0.115473,-0.124302,-0.124388,-0.14072,-0.137716
Peak_Date,2024-09-30 00:00:00,2024-09-30 00:00:00,2024-09-30 00:00:00,2024-09-30 00:00:00,2024-09-30 00:00:00
Trough_Date,2020-03-31 00:00:00,2022-09-30 00:00:00,2022-09-30 00:00:00,2022-09-30 00:00:00,2022-09-30 00:00:00
Recovery_Date,2020-08-31 00:00:00,2024-02-29 00:00:00,2024-02-29 00:00:00,2024-07-31 00:00:00,2024-02-29 00:00:00
